In [ ]:
# Depricated from Google Colab as of 2024-12-01

# !sudo apt install python3.9-full python3-pip
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1

# !pip uninstall -y scikit-learn scipy pyparsing -y
# !pip install Cython==0.29.36 scipy==1.9 pyparsing==2.4
# !pip install https://files.pythonhosted.org/packages/04/e2/b43d4205124dd4c1f14606b2e2d78303db993c6653a90bf11dd0ffe23b5b/scikit_learn-0.24.2-cp39-cp39-manylinux2010_x86_64.whl --no-build-isolation


In [2]:
import pandas as pd
import pickle
import autosklearn
from autosklearn.experimental.askl2 import AutoSklearn2Classifier

from sklearn.model_selection import train_test_split
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
import sklearn.datasets
import sklearn.metrics

import autosklearn.classification
import autosklearn.metrics

from smac.optimizer.smbo import SMBO
from smac.runhistory.runhistory import RunInfo, RunValue

import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [ ]:
import autosklearn

## Import features generated from previous notebook
features = pd.read_csv('features.csv')

features = features[features['id'].isin(train_ids['gma_id']) | features['id'].isin(val_ids['gma_id'])]
test_features = features[features['id'].isin(test_ids['gma_id'])]
holdout_features = features[features['id'].isin(holdout_ids['gma_id'])]


pivot_df = features.pivot_table(index='infant', columns=['feature'], values='Value', fill_value=0)
risk_df = features[['infant', 'risk']].drop_duplicates()

test_pivot_df = test_features.pivot_table(index='infant', columns=['feature'], values='Value', fill_value=0)
test_risk_df = test_features[['infant', 'risk']].drop_duplicates()

holdout_pivot_df = holdout_features.pivot_table(index='infant', columns=['feature'], values='Value', fill_value=0)
holdout_risk_df = holdout_features[['infant', 'risk']].drop_duplicates()

train_df = pd.merge(pivot_df, risk_df, on='infant', how='inner')
test_df = pd.merge(test_pivot_df, test_risk_df, on='infant', how='inner')
holdout_df = pd.merge(holdout_pivot_df, holdout_risk_df, on='infant', how='inner')

X = train_df.drop(columns=['infant', 'risk'])
y = train_df['risk']

X_test = test_df.drop(columns=['infant', 'risk'])
y_test = test_df['risk']

X_holdout = holdout_df.drop(columns=['infant', 'risk'])
y_holdout = holdout_df['risk']

In [ ]:
automl = AutoSklearn2Classifier(
    ensemble_size=1,
    dataset_compression=False,
    allow_string_features=False,
    time_left_for_this_task=300,
    per_run_time_limit=30,
    metric=autosklearn.metrics.balanced_accuracy,
    delete_tmp_folder_after_terminate=False,
    memory_limit=None,
    disable_evaluator_output=False,
)

automl.fit(X, y)
probabilities = automl.predict_proba(X_holdout)[:, 1]

In [ ]:
!wget --content-disposition --max-redirect=20 https://osf.io/download/xa43g/
model = pickle.load(open('automl_vanilla_gma_prediction.pkl', 'rb'))
